<a href="https://colab.research.google.com/github/giuliocapecchi/IR_project/blob/main/IR_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%pip install torch matplotlib nltk tqdm gdown ir_datasets humanize

Note: you may need to restart the kernel to use updated packages.


# 1. Download and prepare the collection

In [12]:
# chosen_collection can be one of ["vaswani", "MSMARCO"]

chosen_collection = "vaswani"

In [13]:
import gdown
import ir_datasets
import pandas as pd
import os

if chosen_collection not in ["vaswani", "MSMARCO"]:
    raise ValueError("chosen_collection must be one of ['vaswani', 'MSMARCO']")

if chosen_collection == "MSMARCO":

    url_collection = 'https://drive.google.com/uc?id=1_wXJjiwdgc9Kpt7o7atP8oWe-U4Z56hn'
    url_stats = 'https://drive.google.com/uc?id=1ruBLKGkKvfFlPMdjVUGkR09_f-528fEz'
    url_lex = 'https://drive.google.com/uc?id=151vGXMMwslHM-Vv8vKRATNLazyAXDM1A'
    url_inv = 'https://drive.google.com/uc?id=1UUL0SkjFq4V9tNGiabRo27GfH3XsUuyk'
    url_doc = 'https://drive.google.com/uc?id=1p-AChVgbUN4nIzFO55-pm8CtCAghobWC'
    
    if not os.path.exists('MSMARCO.tsv'):
        gdown.download(url_collection, 'MSMARCO.tsv', quiet=False)
    
    """os.makedirs('./pickles', exist_ok=True)
    if not os.path.exists('./pickles/stats.pkl'):
        gdown.download(url_stats, './pickles/stats.pkl', quiet=False)
    if not os.path.exists('./pickles/lex.pkl'):
        gdown.download(url_lex, './pickles/lex.pkl', quiet=False)
    if not os.path.exists('./pickles/inv.pkl'):
        gdown.download(url_inv, './pickles/inv.pkl', quiet=False)
    if not os.path.exists('./pickles/doc.pkl'):
        gdown.download(url_doc, './pickles/doc.pkl', quiet=False)"""

elif chosen_collection == "vaswani":
    vaswani_dataset = ir_datasets.load("vaswani")
    docs = list(vaswani_dataset.docs_iter())
    df = pd.DataFrame(docs)
    df['doc_id'] = (df['doc_id'].astype(int) - 1).astype(str)
    # rimuovi i \n da ogni documento
    df['text'] = df['text'].str.replace('\n', ' ')
    df.to_csv('vaswani.tsv', sep='\t', header=False, index=False)

In [14]:
import re
import string
import nltk
import Stemmer # PyStemmer


nltk.download("stopwords", quiet=True)
STOPWORDS = set(nltk.corpus.stopwords.words("english"))
STEMMER = Stemmer.Stemmer('english')

def preprocess(s):
    # lowercasing
    s = s.lower()
    # ampersand and special chars
    s = re.sub(r"[‘’´“”–-]", "'", s.replace("&", " and ")) # this replaces & with 'and' and normalises quotes
    # acronyms
    s = re.sub(r"\.(?!(\S[^. ])|\d)", "", s) # this removes dots that are not part of an acronym
    # remove punctuation
    s = s.translate(str.maketrans(string.punctuation, " " * len(string.punctuation)))
    # strip whitespaces
    s = re.sub(r"\s+", " ", s).strip()
    # tokenisation
    tokens = [t for t in s.split() if t not in STOPWORDS]
    # stemming
    return STEMMER.stemWords(tokens)

In [15]:
import time

def profile(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        ms = (end - start) * 1000
        print(f"{f.__name__} ({ms:.3f} ms)")
        return result
    return f_timer

In [16]:
import pickle
import humanize
import os
from tqdm import tqdm

def print_pickled_size(var_name, var):
    # If the 'tmp' directory does not exist, we first create it
    os.makedirs('./tmp', exist_ok=True)
    with open(f"./tmp/{var_name}.pickle", 'wb') as f:
        pickle.dump(var, f)
    print(f'{var_name} requires {humanize.naturalsize(os.path.getsize(f"./tmp/{var_name}.pickle"))}')
    os.remove(f"./tmp/{var_name}.pickle")
    os.removedirs('./tmp')


def vbyte_encode(number):
    bytes_list = bytearray()
    while True:
        byte = number & 0x7F # Prendi i 7 bit meno significativi -> 0111 1111 = 0x7F
        number >>= 7 # Shifta a destra di 7 bit
        if number:
            bytes_list.append(byte) # Aggiungo i 7 bit al risultato
        else:
            bytes_list.append(0x80 | byte) # Aggiungo i 7 bit con il bit di continuazione, 0x80 = 1000 0000
            break
    return bytes(bytes_list)

def vbyte_decode(bytes_seq):
    number = 0
    for i, byte in enumerate(bytes_seq):
        number |= (byte & 0x7F) << (7 * i)
        if byte & 0x80:
            break
    return number

def decode_concatenated_vbyte(encoded_bytes):
    decoded_numbers = []
    current_number = 0
    shift_amount = 0
    
    for byte in encoded_bytes:
        if byte & 0x80:  # Bit di continuazione trovato, fine del numero
            current_number |= (byte & 0x7F) << shift_amount
            decoded_numbers.append(current_number)
            current_number = 0
            shift_amount = 0
        else:  # Continuo a comporre il numero
            current_number |= (byte & 0x7F) << shift_amount
            shift_amount += 7
    
    return decoded_numbers

#------------------------------------------------------------------------------------------------------------------------------------------------------------------#

def compress_index(lexicon, inv_d, inv_f):    
    compressed_inv_d = {}
    compressed_inv_f = {}

    for term, (termid, df, _) in tqdm(lexicon.items(), desc="Compressing lists", unit="term"):
        encoded_d = bytearray()
        for x in inv_d[termid]:
            encoded_d.extend(vbyte_encode(x)) 
        assert decode_concatenated_vbyte(encoded_d) == inv_d[termid]
        compressed_inv_d[termid] = encoded_d

        encoded_f = bytearray()
        for x in inv_f[termid]:
            encoded_f.extend(vbyte_encode(x))
        assert decode_concatenated_vbyte(encoded_f) == inv_f[termid]
        compressed_inv_f[termid] = encoded_f

    return compressed_inv_d, compressed_inv_f

In [17]:
import pandas as pd
from collections import Counter
from tqdm.auto import tqdm

def build_index(filepath, batch_size=10000):
    total_documents = sum(1 for _ in open(filepath)) # get total number of documents

    lexicon = {}
    inv_d = {}
    inv_f = {}
    doc_index = []
    total_dl = 0
    num_docs = 0
    termid = 0

    with open(filepath, 'r') as file:        
        batch = []
        
        with tqdm(total=total_documents, desc="Processing documents", unit="doc") as pbar:
            for line in file:
                batch.append(line.strip())
                
                # when the batch is full, we process it
                if len(batch) >= batch_size:
                    for line in batch:
                        doc_id, text = line.split('\t', 1) # '1' specifies the number of splits
                        doc_id = int(doc_id)
                        tokens = preprocess(text)
                        token_tf = Counter(tokens)

                        for token, tf in token_tf.items():
                            if token not in lexicon:
                                lexicon[token] = [termid, 0, 0] # termid, df, tf
                                inv_d[termid], inv_f[termid] = [], [] # docids, freqs
                                termid += 1
                            token_id = lexicon[token][0]  # get termid
                            inv_d[token_id].append(doc_id)  # add doc_id to the list of documents containing the term
                            inv_f[token_id].append(tf)  # add term frequency for this doc
                            lexicon[token][1] += 1  # increment document frequency (df)
                            lexicon[token][2] += tf  # increment total term frequency (tf)

                        doclen = len(tokens)
                        doc_index.append((str(doc_id), doclen))
                        total_dl += doclen
                        num_docs += 1                    
                    # update progress bar for each processed document
                    pbar.update(len(batch))
                    batch = []

            # process the remaining documents in the last batch
            if batch:
                for line in batch:
                    doc_id, text = line.split('\t', 1)
                    doc_id = int(doc_id)
                    tokens = preprocess(text)
                    token_tf = Counter(tokens)

                    for token, tf in token_tf.items():
                        if token not in lexicon:
                            lexicon[token] = [termid, 0, 0]
                            inv_d[termid], inv_f[termid] = [], []
                            termid += 1
                        token_id = lexicon[token][0]  # get termid
                        inv_d[token_id].append(doc_id)  # get doc_id to the list of documents containing the term
                        inv_f[token_id].append(tf)  # get term frequency for this doc
                        lexicon[token][1] += 1  # increment document frequency (df)
                        lexicon[token][2] += tf  # increment total term frequency (tf)

                    doclen = len(tokens)
                    doc_index.append((str(doc_id), doclen))
                    total_dl += doclen
                    num_docs += 1                    
                    pbar.update(1)
                    
    stats = {
        'num_docs': num_docs,
        'num_terms': len(lexicon),
        'num_tokens': total_dl,
    }
    return lexicon, {'docids': inv_d, 'freqs': inv_f}, doc_index, stats

In [18]:
import math

class InvertedIndex:

    class PostingListIterator:
        def __init__(self, docids, freqs, doc):
            self.docids = docids
            self.freqs = freqs
            self.pos = 0
            self.doc = doc

        def docid(self):
            if self.is_end_list():
                return math.inf
            return self.docids[self.pos]

        def score(self):
            if self.is_end_list():
                return math.inf
            return self.freqs[self.pos]/self.doc[self.docid()][1]

        def next(self, target = None):
            if not target:
                if not self.is_end_list():
                    self.pos += 1
            else:
                if target > self.docid():
                    try:
                        self.pos = self.docids.index(target, self.pos)
                    except ValueError:
                        self.pos = len(self.docids)

        def is_end_list(self):
            return self.pos == len(self.docids)


        def len(self):
            return len(self.docids)


    def __init__(self, lex, inv, doc, stats):
        self.lexicon = lex
        self.inv = inv
        self.doc = doc
        self.stats = stats

    def num_docs(self):
        return self.stats['num_docs']

    def get_posting(self, termid):
        # extract the encoded docids and freqs
        docids_encoded = self.inv['docids'][termid]
        freqs_encoded = self.inv['freqs'][termid]
        # decode the docids and freqs before returning the iterator
        docids = decode_concatenated_vbyte(docids_encoded)
        freqs = decode_concatenated_vbyte(freqs_encoded)
    
        return InvertedIndex.PostingListIterator(docids, freqs, self.doc)
    

    def get_termids(self, tokens):
        return [self.lexicon[token][0] for token in tokens if token in self.lexicon]

    def get_postings(self, termids):
        return [self.get_posting(termid) for termid in termids]

In [19]:
# import cProfile
# import pstats

# cProfile.run("build_index('./vaswani.tsv')", "output.prof")
# p = pstats.Stats("output.prof")
# p.sort_stats("cumtime").print_stats(10)
# os.remove("output.prof")

Now build up the index for the chosen collection. It is built only if a pickled version of its components doesn't exist already :

In [20]:
import pickle
import os

# If the 'pickles' directory does not exist, we first create it
os.makedirs('./pickles', exist_ok=True)

try: # try to open the pickled files, else build the index
    with open('./pickles/inv_index.pkl', 'rb') as f:
        inv_index = pickle.load(f)
    
except FileNotFoundError:
    lex, inv, doc, stats = build_index(chosen_collection + '.tsv')
    
    # Compress the inverted lists
    inv['docids'], inv['freqs'] = compress_index(lex, inv['docids'], inv['freqs'])
    
    inv_index = InvertedIndex(lex, inv, doc, stats)
    with open('./pickles/inv_index.pkl', 'wb') as f:
        pickle.dump(inv_index, f)


print(f"Numero di documenti: {stats['num_docs']}")
print(f"Numero di termini nel lessico: {stats['num_terms']}")
print(f"Totale token: {stats['num_tokens']}")

Processing documents:   0%|          | 0/11429 [00:00<?, ?doc/s]

Compressing lists:   0%|          | 0/7857 [00:00<?, ?term/s]

Numero di documenti: 11429
Numero di termini nel lessico: 7857
Totale token: 286322


In [21]:
inv_index = InvertedIndex(lex, inv, doc, stats)
print_pickled_size('inv_index', inv_index)

inv_index requires 1.2 MB


In [22]:
# print some postings
termids = inv_index.get_termids(["cat"])
postings = inv_index.get_postings(termids)
for posting in postings:
    print(f"Termid: {posting.docid()}")
    while not posting.is_end_list():
        print(f"Docid: {posting.docid()} - Score: {posting.score()}")
        posting.next()


Termid: 4127
Docid: 4127 - Score: 0.07142857142857142


In [23]:
os.remove('./pickles/inv_index.pkl')

# 2. Download and prepare queries

In [24]:
if chosen_collection == "MSMARCO":
    if not os.path.exists('queries.tar.gz'):
        url = 'https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz'
        gdown.download(url, 'queries.tar.gz', quiet=False)
        !tar -xzf queries.tar.gz
    queries = pd.read_csv('queries.eval.tsv', sep='\t', header=None)
    print("Number of queries: ",len(queries))
elif chosen_collection == "vaswani":
    # Converte le query in un DataFrame
    queries = pd.DataFrame(vaswani_dataset.queries_iter())
    queries.columns = ['qid', 'query']
    print(queries.head(2))
    print("Number of queries: ",len(list(vaswani_dataset.queries_iter()))) 

  qid                                              query
0   1  MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS ...
1   2  MATHEMATICAL ANALYSIS AND DESIGN DETAILS OF WA...
Number of queries:  93


In [25]:
print(queries.head(5))

  qid                                              query
0   1  MEASUREMENT OF DIELECTRIC CONSTANT OF LIQUIDS ...
1   2  MATHEMATICAL ANALYSIS AND DESIGN DETAILS OF WA...
2   3  USE OF DIGITAL COMPUTERS IN THE DESIGN OF BAND...
3   4  SYSTEMS OF DATA CODING FOR INFORMATION TRANSFER\n
4   5  USE OF PROGRAMS IN ENGINEERING TESTING OF COMP...


In [26]:
from collections import namedtuple


class MSMarcoQueries:
    def __init__(self, df):
        self.queries = [Query(row.query_id, row.text) for row in df.itertuples()]

    def queries_iter(self):
        return iter(self.queries)

    def queries_count(self):
        return len(self.queries)


Query = namedtuple('Query', ['query_id', 'text'])


queries.columns = ['query_id', 'text']
queries_dataset = MSMarcoQueries(queries)
print("The number of queries is: ", queries_dataset.queries_count())

The number of queries is:  93


Let's prepare the functions necessary to perform TAAT and DAAT query processing

First, we need a TopQueue class, which stores the top  K  (score, docid) tuples, using an heap 

In [27]:
import heapq

class TopQueue:
    def __init__(self, k=10, threshold=0.0):
        self.queue = []
        self.k = k
        self.threshold = threshold

    def size(self):
        return len(self.queue)

    def would_enter(self, score):
        return score > self.threshold

    def clear(self, new_threshold=None):
        self.queue = []
        if new_threshold:
            self.threshold = new_threshold

    def __repr__(self):
        return f'<{self.size()} items, th={self.threshold} {self.queue}'

    def insert(self, docid, score):
        if score > self.threshold:
            if self.size() >= self.k:
                heapq.heapreplace(self.queue, (score, docid))
            else:
                heapq.heappush(self.queue, (score, docid))
            if self.size() >= self.k:
                self.threshold = max(self.threshold, self.queue[0][0])
            return True
        return False

### TAAT

In [28]:
from collections import defaultdict

def taat(postings, k=10):
    A = defaultdict(float)
    for posting in postings:
        current_docid = posting.docid()
        while current_docid != math.inf:
            A[current_docid] += posting.score()
            posting.next()
            current_docid = posting.docid()
    top = TopQueue(k)
    for docid, score in A.items():
        top.insert(docid, score)
    return sorted(top.queue, reverse=True)

def query_process(query, index):
    qtokens = set(preprocess(query))
    qtermids = index.get_termids(qtokens)
    postings = index.get_postings(qtermids)
    return taat(postings)

### DAAT

In [29]:
import math

def min_docid(postings):
    min_docid = math.inf
    for p in postings:
        if not p.is_end_list():
            min_docid = min(p.docid(), min_docid)
    return min_docid

def daat(postings, k=10):
    top = TopQueue(k)
    current_docid = min_docid(postings)
    while current_docid != math.inf:
        score = 0
        next_docid = math.inf
        for posting in postings:
            if posting.docid() == current_docid:
                score += posting.score()
                posting.next()
            if not posting.is_end_list():
                next_docid = posting.docid()
        top.insert(current_docid, score)
        current_docid = next_docid
    return sorted(top.queue, reverse=True)

def query_process(query, index):
    qtokens = set(preprocess(query))
    qtermids = index.get_termids(qtokens)
    postings = index.get_postings(qtermids)
    return daat(postings)

In [30]:
@profile
def query_processing(queries_iter, fn):
    for q in queries_iter:
        query = preprocess(q.text)
        termids = inv_index.get_termids(query)
        postings = inv_index.get_postings(termids)
        res = fn(postings)

In [31]:
query_processing(queries_dataset.queries_iter(), taat)

query_processing (655.375 ms)


In [32]:
query_processing(queries_dataset.queries_iter(), daat)

query_processing (2058.921 ms)
